In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import pandas as pd
import os

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

hf_token = os.environ.get('hf_token')

# Need auth token for these
hf_auth = hf_token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded on cuda:0


In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [5]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Given a n3.pcap file containing network traffic data, create a Python script using scapy library to process the PCAP. The script should extract the source IP, destination IP, source port, destination port, and the raw payload data for each packet in the PCAP. Store this data in a pandas DataFrame and display it."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1012)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Given a n3.pcap file containing network traffic data, create a Python script using scapy library to process the PCAP. The script should extract the source IP, destination IP, source port, destination port, and the raw payload data for each packet in the PCAP. Store this data in a pandas DataFrame and display it. [/INST]  Sure! Here is a Python script using the Scapy library to process a PCAP file and extract the source IP, destination IP, source port, destination port, and raw payload data for each packet. The data is then stored in a pandas DataFrame and displayed.
```
import scapy
import pandas as pd

# Open the PCAP file
pcap = scapy.open("n3.pcap")

# Create a list to store the packet data
packet_data = []

# Iterate over each packet in the PCAP file
for packet in pcap:
    # Extract the source IP, destination IP, source port, and destination port
    src_ip = packet.src_ip
    dst_ip = packet.dst_ip
    src_port = packet.sport
    dst_port = packet.dport
    
    # Extra

In [6]:
# Run text generation pipeline with our next model
prompt = "Perform BGP analysis using PyBGPStream and detect anomalies in AS path lengths for IPv4 prefixes over two time periods: from January 15, 2020, 15:00 to January 15, 2020, 17:00, and January 18, 2020, 12:00 to January 18, 2020, 13:00."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=718)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Perform BGP analysis using PyBGPStream and detect anomalies in AS path lengths for IPv4 prefixes over two time periods: from January 15, 2020, 15:00 to January 15, 2020, 17:00, and January 18, 2020, 12:00 to January 18, 2020, 13:00. [/INST]  Sure, here's an example of how you could use PyBGPStream to perform BGP analysis and detect anomalies in AS path lengths for IPv4 prefixes over two time periods:

First, you'll need to install PyBGPStream using pip:
```
pip install pypbgpstream
```
Next, you can use the following code to load the BGP data and perform the analysis:
```
import pypbgpstream

# Load the BGP data for the two time periods
bgp_data_15_17 = pypbgpstream.load_bgp_data('2020-01-15T15:00:00/2020-01-15T17:00:00')
bgp_data_18_13 = pypbgpstream.load_bgp_data('2020-01-18T12:00:00/2020-01-18T13:00:00')

# Extract the IPv4 prefixes and their AS path lengths
ipv4_prefixes = bgp_data_15_17['prefixes']
as_path_lengths_15_17 = [len(prefix['as_path']) for prefix in ipv4_prefix

In [2]:
import pypbgpstream

# Load the BGP data for the two time periods
bgp_data_15_17 = pypbgpstream.load_bgp_data('2020-01-15T15:00:00/2020-01-15T17:00:00')
bgp_data_18_13 = pypbgpstream.load_bgp_data('2020-01-18T12:00:00/2020-01-18T13:00:00')

# Extract the IPv4 prefixes and their AS path lengths
ipv4_prefixes = bgp_data_15_17['prefixes']
as_path_lengths_15_17 = [len(prefix['as_path']) for prefix in ipv4_prefixes]

ipv4_prefixes = bgp_data_18_13['prefixes']
as_path_lengths_18_13 = [len(prefix['as_path']) for prefix in ipv4_prefixes]

# Detect anomalies in AS path lengths
anomaly_threshold = 3  # adjust this threshold as needed

anomalies_15_17 = [prefix for prefix in as_path_lengths_15_17 if prefix > anomaly_threshold]
anomalies_18_13 = [prefix for prefix in as_path_lengths_18_13 if prefix > anomaly_threshold]

# Print the anomalies
print('Anomalies in AS path lengths for IPv4 prefixes on January 15, 2020:')
print(anomalies_15_17)

print('Anomalies in AS path lengths for IPv4 prefixes on January 18, 2020:')
print(anomalies_18_13)

NameError: name 'pypbgpstream' is not defined

# 5G General Knowledge

In [3]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/5G/5G_analysis_multi_choice_v2.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]
n_question = 0

for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"Choose the correct answer. {question}\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
    # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
            if line.startswith("Answer: ") or line.startswith("Correct answer: "):
                match = re.search(r'\b(\w)[\)-]', line)
                if match is not None:
                    parsed_answer = match.group(1)
                    print(f"Parsed_answer: {parsed_answer}")
                    break
                else:
                    parsed_answer = "Couldn't parse the answer"
                    print(f"Parsed_answer: {parsed_answer}")
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What happens to packets larger than the Maximum SDU size in the network?
a) Always dropped
b) Forwarded with increased QoS
c) Implementation specific
d) Always forwarded with no changes

Answer: c) Implementation specific

Explanation: The behavior of packets larger than the Maximum SDU size in the network depends on the specific implementation of the network devices and protocols. Some devices may drop such packets, while others may fragment them or drop them with an error message. Therefore, the correct answer is (c) Implementation specific
Parsed_answer: c
----------------1-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. The Maximum Transfer Unit (MTU) of the IP layer and the Maximum SDU Size:
a) Are directly related
b) Have no relationship
c) Are the same
d) Depend on the GGSN

Answer: b) Have no relationship

Explanation: The Maximum Transfer Unit (MTU) of the IP layer and the Maximum SDU Size (Maximum Service Data Unit) are not directly related. The MTU is a parameter that determines the maximum size of a single IP packet, while the Maximum SDU Size
Parsed_answer: b
----------------2-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Generated Answer: Choose the correct answer. For which reason might the PGW reject an IP-CAN session modification procedure after an SGW failure?
a) If the request includes changing the PDN connection route
b) If the request includes the installation/modification of PCC rules
c) If the PGW has already initiated a new SGW restoration process
d) If the MME/SSGSN has already restored the PDN connection

Answer: b) If the request includes the installation/modification of PCC rules.

Explanation: The PGW might
Parsed_answer: b
----------------27-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What does OMR stand for in the context of the provided text?
Optimized Media Routeing
Operational Management Review
Online Music Recorder
Optical Message Reading
OMR stands for Optimized Media Routeing.
----------------28-----------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Choose the correct answer. What protocol is mentioned alongside SIP for OMR based signaling flows?
HTTP
SMTP
SDP
UDP

Answer: B. SDP

Explanation: SIP (Session Initiation Protocol) is a signaling protocol used for initiating, modifying, and terminating sessions in an IP network. OMR (Online Media Resource) based signaling flows use SIP along with SDP (Session Description Protocol) for describing and negotiating the media sessions. HTTP and SMTP are not related to OMR based signaling flows. UDP
Parsed_answer: Couldn't parse the answer
----------------29-----------------
Generated Answer: Choose the correct answer. What is the role of IBCF in the provided context?
Allocating MR context
Decoding audio files
Compressing video streams
Routing email traffic
IBCF (Intelligent Buffer Control Function) is responsible for allocating MR (Multi-Rate) contexts in the provided context. Therefore, the correct answer is:
Allocating MR context.
----------------30-----------------
['c'

In [ ]:
import json
input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_SPEC_QA.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

# 5G analysis knowledge

## Multiple Choice
`5G configuration analysis v1`:
Accuracy: 90%
Correct answers: 21 + 7 = 28
Incorrect answers: 2

`5G configuration analysis v2`:
Accuracy: 70.00%
Correct answers: 21
Incorrect answers: 12